# Configure Pipeline Directory Settings
You must configure the settings before running the encoder pipeline. Paths are already relative to "/content/drive/My Drive/", so only include the path from your Google Drive root. (dir_img = test_1 : The test_1 folder in the root of your Drive)

1.   **dir_encoder:** The directory which contains, or will contain, the StyleGAN encoder. Code will automatically be pulled from the repository if the directory doesn't exist.
2.   **dir_img:** The target directory containing valid image files. Pipeline will automatically generate the sub-directories.
3.   **pretrain_model:** Runs train_resnet.py to train model from scratch. Disable for faster results.

When you have the raw images in the dir_img directory and you are ready, select **"Runtime > Run all (Ctrl+F9)"**. You will need to login to your Drive account and provide a key for Block 3 (copy / paste)


In [1]:
# Backend Settings
generator_url = "https://average-face-datasets.s3.amazonaws.com/generate_average.py"
model_url = "https://average-face-datasets.s3.amazonaws.com/finetuned_resnet.h5"
repo_url = "https://github.com/SirTangent/stylegan-encoder.git"

# Main Settings
dir_encoder = "./" #@param {type:"string"}
dir_img = "images-raw" #@param {type:"string"}
pretrain_model = False #@param {type:"boolean"}

# Parsing
gdrive_root = ""
dir_encoder = gdrive_root + dir_encoder
dir_img = gdrive_root + dir_img

_repo_url = '"' + repo_url + '"'
_dir_encoder = '"' + dir_encoder + '"'
_dir_img = '"' + dir_img + '"'
_model_url = '"' + model_url + '"'

In [3]:
# Encoder Bootloader
import os
from os import path

if not path.isdir(dir_encoder):
  print("Encoder directory not found, cloning master repository...")
  !git clone $_repo_url $_dir_encoder

In [ ]:
!python script_fetch.py $generator_url

In [ ]:
cd $dir_encoder

# Start Encoder Pipeline

In [ ]:
if pretrain_model:
  !python train_resnet.py --loop 1 --max_patience 1
else:
  !python script_fetch.py $model_url --out data/finetuned_resnet.h5

In [ ]:
!python align_images.py $_dir_img $_dir_img/images-aligned

In [ ]:
!python encode_images.py --batch_size=1 --output_video=False $_dir_img/images-aligned $_dir_img/images-generated/ $_dir_img/images-latent

In [ ]:
!python generate_average.py $_dir_img/images-latent --dst_dir $_dir_img/images-average --name=out